# Autosuggest

- Proposta similar a da search engine do Google (dado palavras em um input, qual a próxima palavra que mais se encaixa?)
- Uso de dataset de jornais
- Focado para língua inglesa

## Data

- Uso do Dask para ler os dados iniciais
- Uso do Pandas para o dataframe da língua inglesa

In [1]:
import dask.dataframe as dd
import pandas as pd
import glob, re, os

In [2]:
# Path dos dados
path = os.path.join(os.getcwd(), "data")

# Path dos arquivos pré-existentes
all_files = glob.glob(path + "/english_sentences*.csv")

# Deletando arquivos csv pré-existentes
for file in all_files:
    if os.path.exists(file):
        os.remove(file)    

In [3]:
# Lendo os dados do arquivo tsv
ddf = dd.read_csv("data/old-newspaper.tsv", sep="\t")

# Selecionando somente em inglês e somente a coluna Text, pois os outros dados não serão utilizados
df = ddf[ddf["Language"] == "English"]
df = df[["Text"]]

In [4]:
# Limpeza de particoes vazias
# https://stackoverflow.com/questions/47812785/remove-empty-partitions-in-dask
def cull_empty_partitions(df):
    ll = list(df.map_partitions(len).compute())
    df_delayed = df.to_delayed()
    df_delayed_new = list()
    pempty = None
    
    for ix, n in enumerate(ll):
        if 0 == n:
            pempty = df.get_partition(ix)
            
        else:
            df_delayed_new.append(df_delayed[ix])
            
    if pempty is not None:
        df = dd.from_delayed(df_delayed_new, meta=pempty)
        
    return df

df = cull_empty_partitions(df)

In [5]:
# Convertendo em um dataframe do pandas novamente
data = df.compute()

## Preprocessing

- É necessário fazer a limpeza dos dados.
- Há caracteres diversos, possíveis URLs, entre outros que devem ser removidos da frase.

In [6]:
# Removendo URLs
def limpa_url(texto):
    # Regex obtida de https://www.geeksforgeeks.org/python-check-url-string/
    pattern = r"""
        (?i)  # Ignore case.
        \b  # Inicio de palavra.
        (?:
            https?://
        |
            www
            \d{0,3}
            [.]
        |
            [a-z0-9.\-]+
            [.]
            [a-z]{2,4}
            /
        )
        (?:
            [^\s()<>]+
        |
            \(
            (?:
                [^\s()<>]+
            |
                \(
                [^\s()<>]+
                \)
            )*
            \)
        )+
        (?:
            \(
            (?:
                [^\s()<>]+
            |
                \(
                [^\s()<>]+
                \)
            )*
            \)
        |
            [^\s`!()\[\]{};:'\".,<>?«»“”‘’]
        )
    """
    repl = ""
    matcher = re.compile(pattern, re.VERBOSE)
        
    return matcher.sub(repl, texto)

data["Text"] = data["Text"].apply(lambda t: limpa_url(t))

In [7]:
# Removendo caracteres especiais "@, #, $, %, &, *, `"
data["Text"] = data["Text"].apply(lambda t: re.sub(r"[\@\#\$\%\&\*\`]", "", t))

In [8]:
# Fazendo replace de ? e ! por .
# Esses caracteres indicam o fim de uma frase, mudando apenas sua entonação.
data["Text"] = data["Text"].apply(lambda t: t.replace("?", ".").replace("!", "."))

# Removendo pontuações diversas exceto apostrofes e pontos finais
data["Text"] = data["Text"].apply(lambda t: re.sub(r"[^\w\s\'\.]", "", t))

# Replace de apostrofes por traços para conservar palavras como don't e i'm na hora de tokenizar
data["Text"] = data["Text"].apply(lambda t: t.replace("\'", "-"))

In [9]:
# Fazendo o split das frases por .
data["Text"] = data["Text"].apply(lambda t: t.split("."))

# Comando explode para separar as listas em novas linhas do dataframe
data = data.explode("Text")

In [10]:
# Removendo espaços em branco no início e no fim da frase
data["Text"] = data["Text"].apply(lambda t: t.strip())

# Removendo tabs, newlines e espaços em branco em excesso
data["Text"] = data["Text"].apply(lambda t: t.replace("/\s\s+/g", " "))

# Removendo frases que contém números
data["hasNumbers"] = data["Text"].apply(lambda t: any(char.isdigit() for char in t))
data = data[data["hasNumbers"] == False]

# Removendo frases vazias e mantendo frases que possuem entre 7 e 12 palavras
data = data[(data["Text"] != "") & (data["Text"].str.split(" ").str.len() >= 7) & (data["Text"].str.split(" ").str.len() <= 12)]

In [11]:
# Lowercase
data["Text"] = data["Text"].apply(lambda t: t.lower())

# Reset index
data.reset_index(inplace=True)

# Transforming to list
sentences = data["Text"].tolist()

## NLTK

- após processar os dados, utilizamos o NLTK para:
    - Tokenizacao
    - Uso de stopwords
    - Criacao de ngrams

In [12]:
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords

In [13]:
# Lista dos ngramas
bigrams = []
trigrams = []
fourgrams = []

# Stopwords da língua inglesa
stop_words = set(stopwords.words("english"))

# Iterando sobre cada frase
for sentence in sentences:
    # Tokenizando as frases
    words = word_tokenize(sentence)
    
    # Extendendo a lista dos ngramas com as palavras
    bigrams.extend(list(ngrams(words, 2)))
    trigrams.extend(list(ngrams(words, 3)))
    fourgrams.extend(list(ngrams(words, 4)))   

In [14]:
# Remove ngramas que são compostos exclusivamente por stopwords
def remove_stopwords(ngram: list):
    new_ngram = []
    for sequence in ngram:
        count = 0
        for word in sequence:
            count = count or 0 if word in stop_words else count or 1
        if count == 1:
            new_ngram.append(sequence)
            
    return new_ngram

bigrams = remove_stopwords(bigrams)
trigrams = remove_stopwords(trigrams)
fourgrams = remove_stopwords(fourgrams)

## Probabilidades e Smoothing

Probabilidade de uma palavra no texto:

$\large{P(palavra) = \frac{n_{ocorrencias}}{N_{palavras}}}$

Se a palavra não existe, precisamos adicionar artificialmente para não gerar probabilidade zero. Portanto, adiciona-se 1 (Laplace Smoothing) a todos os membros do vocabulário. Como adiciona-se 1, apenas precisamos contar quantas palavras únicas temos no dicionário para considerar os 1 somados.

$\large{P(palavra) = \frac{n_{ocorrencias} + 1}{N_{palavras} + N_{unicas}}}$

In [15]:
from collections import Counter

n2, n3, n4 = Counter(bigrams), Counter(trigrams), Counter(fourgrams)
s2, s3, s4 = set(bigrams), set(trigrams), set(fourgrams)

In [16]:
# Utilizando a fórmula de probabilidades
p2 = [(n, (n2[n] + 1)/(len(n2) + len(s2))) for n in s2]
p3 = [(n, (n3[n] + 1)/(len(n3) + len(s3))) for n in s3]
p4 = [(n, (n4[n] + 1)/(len(n4) + len(s4))) for n in s4]

# Ordena-se as listas de acordo com a probabilidade de cada N-grama em ordem descrescente
p2 = sorted(p2, key=lambda w: w[1], reverse=True)
p3 = sorted(p3, key=lambda w: w[1], reverse=True)
p4 = sorted(p4, key=lambda w: w[1], reverse=True)

## Resultados

In [17]:
def return_prediction(inp, n, model=2):
    # Lista de possiveis frases
    pred = []
    
    # Tokenize
    inp = word_tokenize(inp)
    
    # Modelo escolhido
    p = None
    
    # Ngram
    ngram = None
    
    if model == 2:
        p = p2
        
    elif model == 3:
        p = p3
                
    elif model == 4:
        p = p4
    
    # Se o tamanho do input for menor que o modelo escolhido, não é possível formar um ngrama.
    # Logo, devemos retornar um erro.
    if len(inp) < model - 1:
        return f"Could not form a n-gram of size {model} with given input."

    # Forma-se os ngramas de tamanho model - 1, e pega-se o último
    ngram = list(ngrams(inp, model - 1))[-1]
    
    count = 0
    
    for wp in p:
        # Pega-se os ngramas do modelo escolhido, remove-se a probabilidade
        # Corta-se a última palavra para tentar descobrir um ngrama que dá match com o nosso ngrama
        if wp[0][:-1] == ngram:
            count += 1
            # Adiciona-se a última palavra, pois houve um match
            
            # O replace serve para caso encontre o -, pois ele foi inserido artificialmente
            # para garantir os casos com apostrofe
            pred.append(wp[0][-1].replace("-", "'"))
            
            # n é o número de sugestões
            if count == n:
                break
                
    # Se não achou, preenche com N/A
    if count < n:
        pred += ["N/A"] * (n - count)
        
    return pred

In [19]:
example = "i want to"
pred = return_prediction(example, n=5, model=3)

for word in pred:
    print(example + " " + word)

i want to be
i want to see
i want to do
i want to go
i want to make


In [20]:
example = "let me see your"
pred = return_prediction(example, n=5, model=4)

for word in pred:
    print(example + " " + word)

let me see your N/A
let me see your N/A
let me see your N/A
let me see your N/A
let me see your N/A


## Futuras iterações

- Trocar para a língua portuguesa
    - Novo preprocessing
    - Novas stopwords


- Input em tempo real


- Modelo com mais N-gramas
    - Retrabalhar o dataset para não gerar muitos cortes


- Frases não são analisadas como um todo, apenas o último n-grama e gera o match


- Faltaria retroalimentação (muitos casos N/A)